### View Steine verschieben L&ouml;sung
- Die Klasse View hat keine Methode `move_stone`, die Methode heisst `move`.

Direkt testen:  
```python
view.on_mouse_down(10, 10) # Klick auf das Feld 0
view.selected_field # 0 (ok)
```

```python
view.on_mouse_up(110, 110) # Klick auf das Feld in Leinwand Mitte
# verursacht Fehler
```


In [16]:
def draw_chessboard(canvas, color1 = 'grey', color2 = 'blue'):
    n = 8 # 8x8 Brett
    # Fuelle Canvas mit color1
    canvas.fill_style = color1
    canvas.fill_rect(0, 0, canvas.width, canvas.height)
    
    # Zeichne (ungerade) Felder mit color2
    canvas.fill_style = color2
    
    # width und height eines Feldes
    dw = canvas.width / n
    dh = canvas.height / n
    
    for x in range(n):
        for y in range(n):
            # zeichen Feld falls x + y gerade
            if (x + y) % 2 == 0:
                canvas.fill_rect(x*dw, y*dh, dw, dh)
                
from ipycanvas import MultiCanvas
from ipywidgets import Output
err_msg = Output(layout = {'border': '1px solid black'})

class View:
    
    def __init__(self, game):
        self.game = game
        # callback registrieren
        game.callback = lambda event, data: self.move_stone(*data)
        
        self.mcanvas = MultiCanvas(3, width = 200, height = 200, 
                                   layout = {'border': '1px solid black'}
                                  )
        self.bg, self.l0, self.l1 = self.mcanvas
        self.layers = {'bg': self.bg, 0: self.l0, 1: self.l1}
        self.layers[0].fill_style = 'red'
        self.layers[1].fill_style = 'yellow'

        self.r = 10
        self.field_size = self.mcanvas.width / 8
        draw_chessboard(self.bg)
        
        # Attribut fuer selektiertes Feld, Callbacks fuer Maus-Events registrieren
        self.selected_field = None
        self.mcanvas.on_mouse_down(self.on_mouse_down)
        self.mcanvas.on_mouse_up(self.on_mouse_up)
       
    def draw_position(self):
        for player, stones in self.game.position.items():
            for idx in stones:
                self.place_stone(player, idx)
            
    def place_stone(self, layer, idx):
        x, y = self.idx2pos(idx)
        self.layers[layer].fill_circle(x, y, self.r)
        
    def remove_stone(self, layer, idx):
        x, y = self.idx2pos(idx)
        self.layers[layer].clear_rect(x-self.r-0.5, y-self.r-0.5, 2*self.r+1)
        
    def move_stone(self, layer, src, target):
        self.remove_stone(layer, src)
        self.place_stone(layer, target)
        
    def idx2pos(self, idx):
        '''gibt x und y Koordinate der Mitte des Feldes mit Nummer idx zurueck'''
        row = idx // 8 
        col = idx % 8  
        return (col+0.5) * self.field_size, (row + 0.5)*self.field_size
      
    def pos2idx(self, pos):
        '''x,y Koordinaten der Klickposition und Feldnummer umrechnen'''
        x, y = pos
        col = int(x // self.field_size)
        row = int(y // self.field_size)
        return 8*row + col   
        
    @err_msg.capture()    
    def on_mouse_down(self, x, y):
        pos = (x, y)
        self.selected_field = self.pos2idx(pos)
        
    @err_msg.capture()    
    def on_mouse_up(self, x, y):   
        if self.selected_field is None:
            return
       
        pos = (x, y)
        target = self.pos2idx(pos)
        # Zug an Game-Instanz weitergeben
        self.game.move(self.game.ptm, self.selected_field, target)  
        self.selected_field = None
        
    def _ipython_display_(self):
        display(self.mcanvas, err_msg)

In [17]:
from game_steine import Game, engine

game = Game()
game.player_name[1] = engine
view = View(game)
view.draw_position()
view

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…